In [196]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '4'

In [197]:
import json
import pandas as pd
import numpy as np
import spacy
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity


from scipy import sparse
from fast_pagerank import pagerank
from fast_pagerank import pagerank_power


pd.set_option('display.max_colwidth', None)

In [10]:
from sentence_transformers import SentenceTransformer, InputExample, LoggingHandler, losses, models, util
import torch

In [23]:
train_args = pd.read_pickle('../../data/train_args_with_scores.pkl')

In [24]:
train_topic_with_scored_kps = pd.read_pickle('../../data/train_topic_keypoints_with_scores.pkl')

In [25]:
train_topics = train_args.groupby(['topic', 'stance']).agg({
    'sents_with_scores': lambda x: set([item for items in x for item in items]),
    'gt-kps' : lambda x: set([item for items in x for item in items]),
    #'gt-cand-kps': lambda x: set([item for items in x for item in items])
}).reset_index()

In [26]:
train_topics.columns

Index(['topic', 'stance', 'sents_with_scores', 'gt-kps'], dtype='object')

In [27]:
train_topics.head()

topic  stance  \
0  Assisted suicide should be a criminal offence      -1   
1  Assisted suicide should be a criminal offence       1   
2                 Homeschooling should be banned      -1   
3                 Homeschooling should be banned       1   
4        The vow of celibacy should be abandoned      -1   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [28]:
#TODO: Check if the gt-kps appears inside sents_with_scores correctly, it seems like something wrong there..

In [29]:
#Add to the candidate sentences from the topic, the topic key-points
train_topics['sents_with_scores'] = train_topics.apply(lambda row: list(row['sents_with_scores']) + train_topic_with_scored_kps[(train_topic_with_scored_kps.topic == row['topic']) & (train_topic_with_scored_kps.stance == row['stance'])]['scored_kps'].tolist()[0], axis=1)

In [30]:
model = SentenceTransformer('/workspace/ceph_data/keypoint-analysis-sharedtask/siamese-models/roberta-base-contrastive-10-epochs-2021-06-08_13-05-35')

In [198]:
def gen_match_matrix(model, sents, topic, min_match_score=0):
    sents1 = [topic + ' <SEP> ' + x for x in sents]
    sents1_embeddings = model.encode(sents1)
    sents_embeddings  = model.encode(sents)
    
    sim_matrix = cosine_similarity(sents1_embeddings, sents1_embeddings)
    super_threshold_indices = sim_matrix < min_match_score
    sim_matrix[super_threshold_indices] = 0

    return sim_matrix

def evalute_topic(row):
    ranked_sents = [x[0] for x in row['ranked_sents']]
    #ranked_sents = [x for x in row['filtered_sents']]
    gt_kps_ranks = []
    for real_kp in row['gt-kps']:
        if real_kp in ranked_sents:
            gt_kps_ranks.append(ranked_sents.index(real_kp)+1)
        else:
            print('kp is not there...')
            gt_kps_ranks.append(len(ranked_sents)+1)
            
    #print(gt_kps_ranks)
    return 1/(sum(gt_kps_ranks)/len(gt_kps_ranks)) if len(gt_kps_ranks) > 0 else 0

In [199]:
def apply_page_rank(row, p=0.85, min_quality_score=0.5, min_match_score=0.5, min_len=20):
    cand_sents = [x for x in row['sents_with_scores'] if x[1] > min_quality_score and len(x[0].split()) < min_len]
    if len(cand_sents) == 0:
        return []
        print('empty')
    cands, cands_qualities = zip(*cand_sents)
    #print(cands)
    #print(cands_qualities)
    cands_matching_mat = gen_match_matrix(model, cands, row['topic'], min_match_score=min_match_score)
    #pr=pagerank(cands_matching_mat, personalize=np.array(cands_qualities), p=p) #it looks like modifying the initial probability doesn't help
    pr=pagerank(cands_matching_mat, p=p)
    ranked_candidates = list(zip(cands, pr))
    return sorted(ranked_candidates, key=lambda x: -x[1])    

In [200]:
def filter_ranked_list(row, model, min_match=0.8):
    ranked_sents = [x[0] for x in row['ranked_sents']]
    filtered_sents = []
    for i, s in enumerate(ranked_sents):
        if len(filtered_sents) == 0:
            filtered_sents.append(s)
            print('FFFF ' , s)
        else:
            matching_scores = gen_match_matrix(model, [s]+filtered_sents, row['topic'])
            max_sim = np.max(matching_scores[0][1:])
            print(matching_scores)
            if max_sim < min_match:
                print('SSSS ' , s)
                filtered_sents.append(s)
    return filtered_sents

In [ ]:
scores = []
for min_match_score in [0.2, 0.4, 0.6, 0.8, 1.0]:
    for p in [0.2, 0.4, 0.8, 1.0]:
        for min_quality_score in [0.2, 0.5, 0.8]:
            train_topics['ranked_sents'] = train_topics.apply(lambda row: apply_page_rank(row, p=p, min_quality_score=min_quality_score, min_match_score=min_match_score), axis=1)
            train_topics['scores'] = train_topics.apply(lambda row: evalute_topic(row), axis=1)
            score = np.mean(train_topics.scores.tolist())
            scores.append([p, min_quality_score, min_match_score, score])

/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseE

kp is not there...


/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseE

kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not th

/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseE

...

/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseE

kp is not there...


/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseE

kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not there...
kp is not th

/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseE

In [ ]:
scores_df = pd.DataFrame(scores, columns=['p', 'min-quality-score', 'min-match-score', 'score'])
scores_df.sort_values('score', ascending=False)

In [137]:
#train_topics['filtered_sents'] = train_topics.apply(lambda row: filter_ranked_list(row, model, min_match=0.3), axis=1)

-------------

##### Apply on test:

In [175]:
test_args = pd.read_pickle('/workspace/ceph_data/keypoint-analysis-sharedtask/siamese-data/data-with-scores/test_args_with_scores.pkl')

In [176]:
test_args.head()

,arg_id,argument,topic,stance,sents,sents_with_scores
0,arg_0_0,Routine child vaccinations isn't mandatory since children don't spread the virus,Routine child vaccinations should be mandatory,-1,[Routine child vaccinations isn't mandatory since children don't spread the virus],"[(Routine child vaccinations isn't mandatory since children don't spread the virus, 0.7141945362091064)]"
1,arg_0_1,Routine child vaccinations should not be mandatory because children may not bear the side effects of it.,Routine child vaccinations should be mandatory,-1,[Routine child vaccinations should not be mandatory because children may not bear the side effects of it.],"[(Routine child vaccinations should not be mandatory because children may not bear the side effects of it., 0.871067464351654)]"
2,arg_0_2,Routine child vaccinations should not be necessary as children can't catch the disease,Routine child vaccinations should be mandatory,-1,[Routine child vaccinations should not be necessary as children can't catch the disease],"[(Routine child vaccinations should not be necessary as children can't catch the disease, 0.8207754492759705)]"
3,arg_0_3,A vaccine that has not been sufficiently tested and without knowledge of side effects is not recommended for children,Routine child vaccinations should be mandatory,-1,[A vaccine that has not been sufficiently tested and without knowledge of side effects is not recommended for children],"[(A vaccine that has not been sufficiently tested and without knowledge of side effects is not recommended for children, 0.8951572775840759)]"
4,arg_0_4,"As long as vaccines are not free of side effects, it cannot make them mandatory for our children.",Routine child vaccinations should be mandatory,-1,"[As long as vaccines are not free of side effects, it cannot make them mandatory for our children.]","[(As long as vaccines are not free of side effects, it cannot make them mandatory for our children., 0.8620935082435608)]"


In [178]:
test_topics = test_args.groupby(['topic', 'stance']).agg({
    'sents_with_scores': lambda x: set([item for items in x for item in items])
}).reset_index()

In [182]:
test_topics['ranked_sents'] = test_topics.apply(lambda row: apply_page_rank(row, p=0.2, min_quality_score=0.5, min_match_score=0.4), axis=1)

/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseE

In [195]:
train_topics['ranked_kps'] = test_topics.apply(lambda row: filter_ranked_list(row, model, min_match=0.5), axis=1)

FFFF  parents should have the freedom to decide what they consider best for their children
[[0.9999999 0.3209411]
 [0.3209411 1.       ]]
SSSS  the poorest people may not be able to pay for the vaccine
[[0.9999997  0.95734894 0.36318588]
 [0.95734894 1.0000001  0.32094115]
 [0.36318588 0.32094115 0.9999999 ]]
[[1.0000002  0.9515432  0.34849045]
 [0.9515432  1.0000001  0.32094115]
 [0.34849045 0.32094115 0.9999999 ]]
[[1.0000002  0.9276775  0.3724013 ]
 [0.9276775  1.0000001  0.32094115]
 [0.3724013  0.32094115 0.9999999 ]]
[[1.         0.93209743 0.33868524]
 [0.93209743 1.0000001  0.32094115]
 [0.33868524 0.32094115 0.9999999 ]]
[[0.9999999  0.32188594 0.4984066 ]
 [0.32188594 1.0000001  0.32094115]
 [0.4984066  0.32094115 0.9999999 ]]
SSSS  Child vaccination shouldn't be mandatory because the virus isn't effective in children
[[1.0000001  0.3313055  0.44408137 0.9154375 ]
 [0.3313055  1.0000001  0.32094115 0.32188594]
 [0.44408137 0.32094115 0.9999999  0.4984066 ]
 [0.9154375  0.3218

KeyboardInterrupt: 

In [185]:
test_topics.head()

topic  stance  \
0                Routine child vaccinations should be mandatory      -1   
1                Routine child vaccinations should be mandatory       1   
2  Social media platforms should be regulated by the government      -1   
3  Social media platforms should be regulated by the government       1   
4                          The USA is a good country to live in      -1   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [ ]:
keypoints_ds = []
c = 0
for i, row in test_topics.iterrows():
    for kp, score in row['ranked_kps'][0:5]:
        keypoints_df.append([c, row['topic'], row['stance'], kp])
        
keypoints_df = pd.DataFrame(keypoints_df, columns=['key_point_id', 'topic', 'stance', 'key_point'])